Import Libraries

In [1]:
import cv2 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

Importing Data

In [2]:
df = pd.read_csv("train_dataset.csv") 

newdf = pd.DataFrame(columns=['id', 'numpy', 'emotion'])

s=[]
pixels = df.iloc[:, 1].to_numpy() #converting the second column to a numpy array

for i in pixels:
    number_list = i.split()
    number_list = [int(num) for num in number_list]
    s.append(number_list)
s=np.array(s)
#s is a 2d array containing all the images in an array format
emotion=df.iloc[:, 2].to_numpy()


#data augmentation
#changing contrast
#changing brightness
counter=0
for i in range(s.shape[0]):
    image = s[i].reshape((48,48))
    brightness = 20 
    contrast = 100
    image2 = cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness) 
    image3 = cv2.flip(image, 1)
    image=image.reshape((2304,))
    image2=image2.reshape((2304,))
    image3=image3.reshape((2304,))
    newdf.loc[counter] = [counter, image, emotion[i]]
    counter+=1
    newdf.loc[counter] = [counter, image2, emotion[i]]
    counter+=1
    newdf.loc[counter] = [counter, image3, emotion[i]]
    counter+=1


id= newdf.iloc[:, 0].to_numpy() #converting the first column of df into a numpy array id
emotion=newdf.iloc[:, 2].to_numpy()
s=[]
for i in range(15000):
    s.append(newdf.iloc[i]['numpy'])
s=np.array(s)#updated s, contianing the array representation of the augmented data as well.


Compiling the Model

In [4]:
tf.random.set_seed(1234)
model = Sequential(
    [               
        
        tf.keras.layers.InputLayer((2304,)),
        tf.keras.layers.Dense(20, activation="relu", name="L1"),
        tf.keras.layers.Dense(10, activation="relu", name="L2"),
        tf.keras.layers.Dense(7, activation="linear", name="L3")
       
    ], name = "my_model" 
)


model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)



Fit the Model

In [5]:
model.fit(
    s,emotion,
    epochs=10
)

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 272.3691
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8625
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8340
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8212
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.8145
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.8105
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.8082
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.8067
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.8058
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8051


In [7]:
dff = pd.read_csv("train_dataset.csv") 
idd= dff.iloc[:, 0].to_numpy() #converting the first column of df into a numpy array id
ss=[]
pixelss = dff.iloc[:, 1].to_numpy() #converting the second column to a numpy array
for i in pixelss:
    number_list = i.split()
    number_list = [int(num) for num in number_list]
    ss.append(number_list)
ss=np.array(ss)
#ss is the numpy array containing the info about all the test images
y_pred = model.predict(ss)
y_pred = y_pred.argmax(axis=1)

#creating a new data-frame which can be converted to csv for submission
fdf=pd.DataFrame()
se = pd.Series(idd)
fdf['id'] = se.values
see=pd.Series(y_pred)
fdf['emotion']=see.values
fdf.to_csv('failed_results.csv',index=False)#I deleted the file that was created after running this, so to see the result of this pls run this block again


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
